In [44]:
import numpy as np
import pandas as pd

In [45]:
credits_df = pd.read_csv('credits.csv')
movies_df = pd.read_csv('movies.csv')

In [46]:
movies_df = movies_df.merge(credits_df, on='title')

In [47]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4808 entries, 0 to 4807
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4808 non-null   int64  
 1   genres                4808 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4808 non-null   int64  
 4   keywords              4808 non-null   object 
 5   original_language     4808 non-null   object 
 6   original_title        4808 non-null   object 
 7   overview              4805 non-null   object 
 8   popularity            4808 non-null   float64
 9   production_companies  4808 non-null   object 
 10  production_countries  4808 non-null   object 
 11  release_date          4807 non-null   object 
 12  revenue               4808 non-null   int64  
 13  runtime               4806 non-null   float64
 14  spoken_languages      4808 non-null   object 
 15  status               

In [48]:
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [49]:
movies_df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [50]:
movies_df.dropna(inplace=True)

In [51]:
movies_df.duplicated().sum()

0

In [52]:
movies_df.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [53]:
import ast

In [54]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [55]:
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)

In [58]:
def convert3(obj):
    L=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [62]:
movies_df['cast'] = movies_df['cast'].apply(convert3)

In [64]:
def fetch_directors(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [65]:
movies_df['crew'] = movies_df['crew'].apply(fetch_directors)

In [67]:
movies_df['overview'] = movies_df['overview'].apply(lambda x: x.split())

In [69]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [70]:
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

In [73]:
df = movies_df[['movie_id', 'title', 'tags']]

In [75]:
df['tags'] = df['tags'].apply(lambda x: ' '.join(x))

C:\Users\gabri\AppData\Local\Temp\ipykernel_2500\1753472850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: ' '.join(x))


In [78]:
df['tags'] = df['tags'].apply(lambda x: x.lower())

C:\Users\gabri\AppData\Local\Temp\ipykernel_2500\4276611149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: x.lower())


In [80]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [81]:
cv.fit_transform(df['tags']).toarray().shape

(4805, 5000)

In [82]:
vectors = cv.fit_transform(df['tags']).toarray()

In [87]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [88]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [89]:
df['tags'] = df['tags'].apply(stem)

C:\Users\gabri\AppData\Local\Temp\ipykernel_2500\866399325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [93]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [96]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6]

[(539, 0.26089696604360174),
 (507, 0.25302403842552984),
 (1192, 0.25226248955475644),
 (1214, 0.2480694691784169),
 (582, 0.24397501823713333)]

In [105]:
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(df.iloc[i[0]].title)

In [108]:
recommend('Avatar')

Titan A.E.
Independence Day
Small Soldiers
Aliens vs Predator: Requiem
Battle: Los Angeles


In [109]:
recommend('Iron Man')

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
Captain America: Civil War
The Avengers


In [110]:
recommend('Liar Liar')

13 Going on 30
Heartbreakers
A Simple Wish
A Good Year
21 & Over


In [111]:
recommend('Captain America: Civil War')

Captain America: The First Avenger
Captain America: The Winter Soldier
Iron Man 3
Avengers: Age of Ultron
Iron Man 2


In [114]:
recommend('Ant-Man')

Captain America: Civil War
Iron Man 3
Iron Man 2
Iron Man
The Avengers
